In [13]:
# importing necessary modules
from pymongo import MongoClient as mc
#from pathlib import Path
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import pandas as pd
import fingerprint_enhancer
from walking import walking, walkonce, checkstable, mergeneighbors
import math

In [14]:
def get_image_paths_and_mv():
    # this funciton only return image paths from "FVC2000" folder
    main_lst = []
    mvs = []
    paths_list = []

    db_client = mc("10.5.18.101")
    coll = db_client.BI.FVC2006
    coll = coll.find({})
    
    for i in coll:
        #if(so_far < count):
        main_lst.append(['../../../19CS91R05/datasets/' + i['path'] , i['mv'] ])
        paths_list.append('../../../19CS91R05/datasets/' + i['path'])
        mvs.append(i['mv'])
    
    return main_lst

In [15]:
def walk_core2(en_image):
	detect_SP = walking(en_image)
	return detect_SP

In [16]:
# Lets check the Minutiae binary pattern

def checkIfPossible(x,y,image_height, image_width):
    if(x<0 or x>=image_height):
        return False
    elif(y<0 or y>=image_width):
        return False
    return True

def GetMinutiaBinaryPattern(minutiae, fingerprint):
    MBP_List = []
    # fingerprint = cv.imread(fingerprint_path, cv.IMREAD_GRAYSCALE)

    image_height, image_width = fingerprint.shape

    for i in minutiae:
        x, y = i[:2]
        window = [[0 for i in range(3)] for j in range(3)]
        if(checkIfPossible(x-1, y+1,image_height,image_width)):
            window[0][0] = fingerprint[x-1][y+1]
        if(checkIfPossible(x, y+1,image_height,image_width)):
            window[0][1] = fingerprint[x][y+1]
        if(checkIfPossible(x+1, y+1,image_height,image_width)):
            window[0][2] = fingerprint[x+1][y+1]
        if(checkIfPossible(x-1, y,image_height,image_width)):
            window[1][0] = fingerprint[x-1][y]
        if(checkIfPossible(x, y,image_height,image_width)):
            window[1][1] = fingerprint[x][y]
        if(checkIfPossible(x+1, y,image_height,image_width)):
            window[1][2] = fingerprint[x+1][y]
        if(checkIfPossible(x-1, y-1,image_height,image_width)):
            window[2][0] = fingerprint[x-1][y-1]
        if(checkIfPossible(x, y-1,image_height,image_width)):
            window[2][1] = fingerprint[x][y-1]
        if(checkIfPossible(x+1, y-1,image_height,image_width)):
            window[2][2] = fingerprint[x+1][y-1]
        
        temp = []
        
        # direction of MBP vector (0,0) -> (0,1) -> (0,2) -> (1,2) -> (2,2) -> (2,1) -> (2,0) -> (1,0)
        # Hard coding

        temp.append(window[0][0]//255)
        temp.append(window[0][1]//255)
        temp.append(window[0][2]//255)
        temp.append(window[1][2]//255)
        temp.append(window[2][2]//255)
        temp.append(window[2][1]//255)
        temp.append(window[2][0]//255)
        temp.append(window[1][0]//255)
        
        
        # for j in range(3):
        #     for k in range(3):
        #         if(k == 1 and j ==1):
        #             continue
        #         else:
        #             temp.append(window[j][k])
        MBP_List.append(temp)
    return MBP_List

In [17]:
def create_data_frame(lst):
    fingerprint_path, minutia = lst
    # global main_df
    img = cv.imread(fingerprint_path,cv.IMREAD_GRAYSCALE)
    en_image = fingerprint_enhancer.enhance_Fingerprint(img)
    # en_image_bw = cv.cvtColor(en_image, cv.COLOR_GRAY2BGR)
    distance = []
    sectors = []
    try:
        SP = walk_core2(en_image)
    except:
        return
    
    if(min(SP['core'].shape) == 0):
        if min(SP['delta'].shape) == 0:
            # NO Singular Point
            return
        else:
            CorePoint = (int(SP['delta'][0,0]), int(SP['delta'][0,1]))
    else:
        CorePoint = (int(SP['core'][0,0]), int(SP['core'][0,1]))

    for i in minutia:

        dis = ((CorePoint[0] - i[0])**2 + (CorePoint[1] - i[1])**2 )**0.5

        distance.append(dis)

        rads = math.atan2((i[1]- CorePoint[1]), (i[0]- CorePoint[0]))
        
        #if CorePoint[0] == i[0]:
        #    rads = 90
        #else:
        #    rads = math.atan((CorePoint[1] - i[1])/(CorePoint[0] - i[0]))


        degs = math.degrees(rads)
        if degs < 0:
            degs = 360 + degs
        
        sectors.append(int(degs//5 + 1 ))
    
    mbps = GetMinutiaBinaryPattern(minutia, en_image)
    pths = [fingerprint_path for i in range(len(minutia))]
    
    data = {
        "Distance" : distance,
        "Sector" : sectors,
        "MBP" : mbps,
        "Image Path" : pths 
    }
    
    new_df = pd.DataFrame(data)
    
    return new_df



In [20]:
from multiprocessing import Pool
main_df = pd.DataFrame(columns= ['Distance', 'Sector', 'MBP', 'Image Path'])
lst = get_image_paths_and_mv()
with Pool(10) as P:
    ans = P.map(create_data_frame, lst)

In [21]:
lst

[]

In [19]:
con = pd.concat(ans, ignore_index=True)
con.to_csv('FVC_2006_index_space.csv')
con

ValueError: No objects to concatenate